<h1 style="font-family:Papyrus; font-size:3em" align="center"> <img src="data/rar.png"  height="45" width="45" > <b>Project №3 'EDA'</b> <img src="data/rar.png"  height="45" width="45" >
</h1>


## 1. Import libs and define constants

In [125]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# импортируем библиотеки для визуализации
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

from category_encoders import OrdinalEncoder

from sklearn.model_selection import train_test_split  
from scipy.stats import shapiro
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели  
from sklearn import metrics
# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [126]:
RANDOM_SEED = 42

In [127]:
!pip freeze > requirements.txt

In [128]:
# Подгрузим наши данные из соревнования
# DATA_DIR = '/kaggle/input/sf-booking/'
DATA_DIR = './data/' # home dir
df_train = pd.read_csv(DATA_DIR+'hotels_train.csv') # датасет для обучения
df_test = pd.read_csv(DATA_DIR+'hotels_test.csv') # датасет для предсказания
# sample_submission = pd.read_csv(DATA_DIR+'/submission.csv') # самбмишн

## 2. First look at hotels data


In [129]:
df_train.head(2)

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,tags,days_since_review,lat,lng
0,Stratton Street Mayfair Westminster Borough Lo...,581,2/19/2016,8.4,The May Fair Hotel,United Kingdom,Leaving,3,1994,Staff were amazing,4,7,10.0,"[' Leisure trip ', ' Couple ', ' Studio Suite ...",531 day,51.507894,-0.143671
1,130 134 Southampton Row Camden London WC1B 5AF...,299,1/12/2017,8.3,Mercure London Bloomsbury Hotel,United Kingdom,poor breakfast,3,1361,location,2,14,6.3,"[' Business trip ', ' Couple ', ' Standard Dou...",203 day,51.521009,-0.123097


In [130]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386803 entries, 0 to 386802
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               386803 non-null  object 
 1   additional_number_of_scoring                386803 non-null  int64  
 2   review_date                                 386803 non-null  object 
 3   average_score                               386803 non-null  float64
 4   hotel_name                                  386803 non-null  object 
 5   reviewer_nationality                        386803 non-null  object 
 6   negative_review                             386803 non-null  object 
 7   review_total_negative_word_counts           386803 non-null  int64  
 8   total_number_of_reviews                     386803 non-null  int64  
 9   positive_review                             386803 non-null  object 
 

In [131]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128935 entries, 0 to 128934
Data columns (total 16 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               128935 non-null  object 
 1   additional_number_of_scoring                128935 non-null  int64  
 2   review_date                                 128935 non-null  object 
 3   average_score                               128935 non-null  float64
 4   hotel_name                                  128935 non-null  object 
 5   reviewer_nationality                        128935 non-null  object 
 6   negative_review                             128935 non-null  object 
 7   review_total_negative_word_counts           128935 non-null  int64  
 8   total_number_of_reviews                     128935 non-null  int64  
 9   positive_review                             128935 non-null  object 
 

Merge train and test data with marks for coop transformations


In [132]:
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['reviewer_score'] = 0 # в тесте у нас нет значения reviewer_score, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

C:\Users\mblyz\AppData\Local\Temp\ipykernel_10884\1295228875.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем


In [133]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515738 entries, 0 to 515737
Data columns (total 18 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               515738 non-null  object 
 1   additional_number_of_scoring                515738 non-null  int64  
 2   review_date                                 515738 non-null  object 
 3   average_score                               515738 non-null  float64
 4   hotel_name                                  515738 non-null  object 
 5   reviewer_nationality                        515738 non-null  object 
 6   negative_review                             515738 non-null  object 
 7   review_total_negative_word_counts           515738 non-null  int64  
 8   total_number_of_reviews                     515738 non-null  int64  
 9   positive_review                             515738 non-null  object 
 

## Now do data clean and EDA


In [134]:
data.drop(['lat', 'lng'], axis=1, inplace=True)

In [135]:
# булевы маски для удобного доступа к train и test данным
train_mask = data['sample'] == 1 
test_mask = data['sample'] == 0

In [136]:
data.drop('hotel_name', axis=1, inplace=True)

Выделим страну из адреса, предварительно посмотрев что имеется 6 уникальных стран, сразу же закодируем в числовой признак и откинем признак адреса

In [137]:
data['country'] = data['hotel_address'].apply(lambda x: x.split()[-1])
data['country'] = (OrdinalEncoder().fit_transform(data['country']))
data.drop('hotel_address', axis=1, inplace=True)

Поработаем с датой обзора, добавим признак времени года в числовом виде

In [138]:
data['review_date'] = pd.to_datetime(data['review_date'])    
data['season'] = data['review_date'].apply(lambda date: date.month%12 //3 + 1)

In [139]:
data['days_since_review'] = data['days_since_review'].apply(lambda x: int(x.split()[0]))

In [140]:
# data['positive_review'].value_counts().nlargest(12)
def positive_score (row):
    row = row.lower()
    if row == 'no positive' or row =='nothing':
        return 0
    if 'location' in row:
        return 2
    if 'everything' in row:
        return 3
    else:
        return 1
data['positive_review'] = data['positive_review'].apply(positive_score)
# data

In [141]:
data.drop('negative_review', axis=1, inplace=True )

In [142]:
data['part_of_additional_reviews'] = data['additional_number_of_scoring'] / data['total_number_of_reviews'] * 100
data.drop('additional_number_of_scoring', axis=1, inplace=True)

Работаем с тегами


In [189]:
def get_nights(tags):
    tags = list(tags[1:-1].replace('\'', '').split(', '))
    flag = False
    for i in range(len(tags)):
        # print(tags[i])
        if 'night' in tags[i] and (tags[i].split()[-2]).isdigit():
            flag = True
            break
    if flag:
        return int(tags[i].split()[-2])
    else:
        return -1
        
    
    
    
# def unique_tags(elem):
#     lst = list(elem.replace('[', '').replace(']','').replace('\'', '').split(', '))
#     for elem in lst:
#         tags_set.add(elem)
#         if elem not in tags_count:
#             tags_count[elem] = 1
#         else:
#             tags_count[elem] += 1
data['tags_count'] = data['tags'].apply(lambda x: len(list(x[1:-1].replace('\'', '').split(', '))))
data['nights_count'] = data['tags'].apply(get_nights)
data[data['nights_count'] == -1]['tags'].to_csv('temp.csv')

## Now divide test and train data

In [ ]:
train_data = data.query('sample == 1').drop(['sample'], axis=1)
test_data = data.query('sample == 0').drop(['sample'], axis=1)

y = train_data.reviewer_score.values            # наш таргет
X = train_data.drop(['reviewer_score'], axis=1)

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

In [ ]:
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
# model.fit(X_train, y_train)

# Предсказанные значения записываем в переменную y_pred
# y_pred = model.predict(X_test)

In [ ]:
print('MAPE:', metrics.mean_absolute_percentage_error(y_test, y_pred))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['reviewer_score'], axis=1)

In [ ]:
predict_submission = model.predict(test_data)
predict_submission

In [ ]:
sample_submission['reviewer_score'] = predict_submission
sample_submission.head(10)
sample_submission.to_csv('submission.csv', index=False)
